How to use gulp?

In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-runtime');
    require.resolve('gulp-replace');
    require.resolve('gulp-tap');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp#4.0" gulp-runtime gulp-replace gulp-tap');
}
var gulp = require('gulp');
var tap = require('gulp-tap');
var replace = require('gulp-replace');

// rename the project
gulp.task('build :src :dest', function (files, project) {
    return gulp
        .src(files, {cwd: project})
        .pipe(replace('appId: \'my-app-id\'', 'appId: \'jupytangular-module-server\''))
        .pipe(tap(function(file) {
            console.log(file.contents.toString());
        }))
});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';

gulp.task('default', () => gulp.task(['build :src :dest'])(['**/app.browser.module.ts'], project));

$$.async()
try {
gulp.series(() => gulp.task(['default'])(), function (done) {
    done();
    $$.sendResult();
})();
} catch (e) {
    $$.sendError(e);
}

How to convert a gulp task to a Promise?

How to use a Promise as a gulp task?



In [ ]:
var gulp = require('gulp');

// I wish Linus Torvolds had absolute power over everything javascript(+derivatives) and he could just do a massive purge of all the bullshit.
var tasksToPromise = (tasks) => {
    try {
        return new Promise((resolve, reject) => {
            gulp.series.apply(gulp, tasks.concat([function (done) {
                resolve(done());
            }]))();
        });
    } catch(e) {
        reject(e);
    }
};
module.exports = tasksToPromise;
tasksToPromise;



In [ ]:
var fs = require('fs');
var path = require('path');
var replace = require('gulp-replace');
var gulp = require('gulp');

// add automock to project for callback services
var client, renderer, server, core;

var applyUniversal = (project) => {
    // gulp.task('theme',
    //    gulp.src('./some/dir/**/*.js', {cwd: 'path/to/project'})
    //        .src('./specific-file.ts')) // already within cwd in chain?

    // TODO: copy auth, resolve dependencies using file system script, generate code below
    
    // replace package.json
    gulp.task('project relatives',
        gulp.series(projectRelatives(project)))
    // in *.js Can't resolve 'socket.io-client/package'
    // in *.ts Cannot find name 'JwtHelper' -> trace back to import statements?
    
    
    gulp.task('server module', function() {
        return gulp.src(['**/server/app/app.server.module.js'], {cwd: project})
            // add mock socket server to server/render-server or app.server.module
        .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from '../../client/app/auth/auth.service';
import { LogService } from '../../imports/log.service';
import { AuthManager } from '../../client/app/auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from '../../client/app/auth/mailgun-validate.service';
`))
   // TODO: pipe server.ts to add log provider for log service    
            .pipe(replace(
             /\)\],\s*AppModule\s*,providers\s*:\s*\[/ig,
              `)],
        AppModule
    ],
    providers: [
        {
            provide: AuthService,
            useValue: require('./sockify-server').sockifyRequire(AuthService, 'AuthService')
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-server').sockifyRequire(PlatformLocation, 'PlatformLocation')
        },
        {
            provide: LogService,
            useValue: require('./sockify-server').sockifyRequire(LogService, 'LogService')
        },
        {
            provide: AuthGuard,
            useValue: require('./sockify-server').sockifyRequire(AuthGuard, 'AuthGuard')
        },
        AuthManager, JwtHelper, MailgunValidatorService,

    `))
            .pipe(gulp.dest(project))
    });
    
    // add socket-client provider for auth module to app.browser.module
    gulp.task('browser module', function() {
      return gulp.src(['**/client/app/app.browser.module.ts'], {cwd: project})
        .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from './auth/auth.service';
import { PlatformLocation } from '@angular/common';
import { LogService } from '../../imports/log.service';
import { AuthManager } from './auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from './auth/mailgun-validate.service';
`))
        .pipe(replace(
          /\)\],\s*AppModule/ig,
          `)],
        AppModule,
    ],
    providers: [ AuthManager, JwtHelper, MailgunValidatorService,
        {
            provide: AuthService,
            useValue: require('./sockify-client').sockifyClient(AuthService, 'AuthService'),
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-client').sockifyClient(PlatformLocation, 'PlatformLocation'),
        },
`))
        .pipe(gulp.dest(project))
    });
    
    // replace routes with lazy modules from other projects
    gulp.task('app router', function() {
      return gulp.src(['**/client/app/app.routes.ts'], {cwd: project})
        .pipe(replace(
         /children: \[/ig, 
          `children: [
            {
                path: 'auth',
                loadChildren: './auth/auth.module#AuthModule'
            },
`))
        .pipe(gulp.dest(project))
    });
    
    // TODO: try new versions of packages to act like greenkeeper.io
    
    // gulp apply everything
    gulp.task('apply universal',
        gulp.series('server module',
                    'browser module'
                    'app router'));
    
    return tasksToPromise(['apply universal'], []);
};
applyUniversal;



Apply universal to angular project?

In [ ]:

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';
if(!fs.existsSync(project)) {
    fs.mkdirSync(project);
}

$$.async()
applyUniversal(project)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Fix project relatives as a service for updating files when they change?



In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-watch');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp.git#4.0" gulp-watch');
}
var gulp = require('gulp'),
    watch = require('gulp-watch');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';

var searching = false;;
gulp.task('watch for changes', function () {
    // Callback mode, useful if any plugin in the pipeline depends on the `end`/`flush` event 
    return watch('src/**/*.ts', {
        cwd: project,
        ignoreInitial: true,
        read: false,
        readDelay: 100
    }, function () {
        if(!searching) {
            searching = true;
            projectRelatives(project).then(() => { searching = false });
        }
    });
});

$$.async();
gulp.task(['watch for changes'])();



Search notebooks for a string using gulp?



In [1]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/jupytangular2';

var gulp = require('gulp');
var tap = require('gulp-tap');
var JSONStream = require('JSONStream');
var Duplex = require('stream').Duplex;

function bufferToStream(buffer) {  
    let stream = new Duplex();
    stream.push(buffer);
    stream.push(null);
    return stream;  
}

var cells;
gulp.task('search notebooks', function () {
    cells = [];
    return gulp.src(['**/*.ipynb', '!**/node_modules/**', '!**/\\.*'], {cwd: project})
        .pipe(tap(file => {
            var matches = bufferToStream(file.contents).pipe(JSONStream.parse([true, {emitPath: true}]));
            matches.on('data', match => {
                if(match.path[0] === 'cells') {
                    if(match.value.source.join('').match(new RegExp('eval\\(', 'ig'))) {
                        cells[cells.length] = file.path + ':\n\n```\n' + match.value.source.join('') + '\n```\n\n';
                    }
                }
            });
        }));
});

$$.async();
gulp.series('search notebooks', function (done) {
    done();
    $$.mime({'text/markdown': cells.join('\n')});
})();


C:\Users\brian.cullinan\Documents\jupytangular2\Analytics\reconcile timeline data.ipynb:

```
var path = require('path');
var calendarPath = '../Utilities/google calendar.ipynb';
var importer = require('../Core');
var placesNearby;
var compareTimeline = (date) => {
    var newDate = new Date(date);
    newDate.setHours(0,0,0);
    var endDate = new Date(newDate);
    endDate.setHours(23, 59, 59);
    return importer.interpretAll(['find a place'])
        .then(r => {
            placesNearby = eval(r[0].code);
        })
        .then((r) => importer.importNotebook(calendarPath))
        .then((r) => r.listEventBetweenTwoDates(newDate, endDate))
        .then(r => r.filter(e => typeof e.event.location !== 'undefined'))
        //.then(r => console.log(r))
        .then(r => Promise.all(r.map(e => {
            var cache = getTimelineCache(date);
            cache.locations.forEach(l => {

            });
            // get nearest location event to date
            return placesNearby();
        })))
}

(compareTimeline);


```


C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb:

```

var testNotebook = 'test import.ipynb';
var testCells = [
    {cell_type: 'code', source: []},
    {cell_type: 'markdown', source: ["how to test for errors line 2?"]},
    {cell_type: 'code', source: ["(function (err) { throw error; })"]}
];
global[testNotebook+'[1]'] 
    = eval(testCells[testCells.length-1].source.join('').trim());
var markdownCache = cacheCells(testCells, testNotebook);
if(markdownCache[0].id == testNotebook+'[1]') {
    $$.sendResult(cacheCells);
}

```


C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\build Angular components.ipynb:

```
var execSync = require('child_process').execSync;
try {
    require.resolve('@angular/cli');
} catch (e) {
    execSync('npm install @angular/cli');
}

// use utility/filesystem to mock all fs and typescript commands
var mockTypescriptFs, cli;

var importer = require('../Core');
// call the CLI just like from command line
var ng = (project, args = ['generate', 'component', 'test']) => {
    // set up project path
    var previous = process.cwd();
    
    var conf = {
        cliArgs: args,
        inputStream: null,
        outputStream: process.stdout
    };
    
    // execute
    return importer.interpretAll('memory-fs rewire')
        .then(r => {
            if(typeof mockTypescriptFs == 'undefined') {
                mockTypescriptFs = eval("'use strict';" + r[0].code);
                // overlay out temp filesystem on top of current filesystem
                mockTypescriptFs(project);
                cli = require('@angular/cli');
            }
            process.chdir(project);
            return cli(conf);
        });
};
ng;


```


C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\build Angular components.ipynb:

```
var execSync = require('child_process').execSync;
    try {
    require.resolve('webpack');
} catch (e) {
    execSync('npm install webpack');
}

// source https://github.com/christianalfoni/webpack-bin/issues/106
var webpackAngularProject = (project) => {
    // set up project path
    var previous = process.cwd();
    process.chdir(project);
    module.paths.unshift(project + '/node_modules');
    
    var webpack = require('webpack');
    var webpackConfig = require(path.join(project, 'webpack.config.js'));
    var gulp = require(path.join(project, 'config', 'gulp-helpers.js'));
    var settings = gulp.loadSettings(require(path.join(project, 'config', 'build-config.json')));
    var config = webpackConfig({
          env: 'prod',
          platform: 'server'
        }, gulp.root, settings);
    var compiler = webpack(config);

    // TODO: do we need this since it is already mocked in angular-cli?
    //mockTypescriptFs = eval("'use strict';" + r[0].code);
    //mockTypescriptFs(project, data);
    //compiler.inputFileSystem = memfs;
    //compiler.outputFileSystem = memfs;
    //compiler.resolvers.normal.fileSystem = memfs;
    //compiler.resolvers.context.fileSystem = memfs;

    return new Promise((resolve, reject) => {
        compiler.run(function(err, stats) {
            process.chdir(previous);
            if (err) return reject(err);
            resolve(stats.compilation.assets['server.js'].source());
        });
    });
};
webpackAngularProject;

```


C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\promisify automock socketio.ipynb:

```
var execSync = require('child_process').execSync;
try {
    require.resolve('bluebird');
    require.resolve('socket.io');
    require.resolve('socket.io-client');
} catch (e) {
    execSync('npm install bluebird automock socket.io socket.io-client');
}

$$.async();
var importer = require('../Core');
var automock;
importer.interpretAll(['mock all properties and functions using rewire'])
    .then(r => {
        automock = eval(r[0].code);
        $$.sendResult('automock loaded');
    });


```


C:\Users\brian.cullinan\Documents\jupytangular2\Marketing\Wireframing.ipynb:

```
const fs = require('fs');
const js1 = fs.readFileSync('./sosmethod/dist/www/polyfills.59868df8094f160adda5.js');
const js2 = fs.readFileSync('./sosmethod/dist/www/vendor.59868df8094f160adda5.js');
const js3 = fs.readFileSync('./sosmethod/dist/www/app.59868df8094f160adda5.js');

$$html$$ = '<bc-app style="position:relative;display:block;min-height:600px;min-width:600px;">hello!</bc-app>' 
    + '<script type="application/javascript">eval(atob("' + (new Buffer(js1)).toString('base64') + '"));</script>'
    + '<script type="application/javascript">eval(atob("' + (new Buffer(js2)).toString('base64') + '"));</script>'
    + '<script type="application/javascript">eval(atob("' + (new Buffer(js3)).toString('base64') + '"));</script>';


```


C:\Users\brian.cullinan\Documents\jupytangular2\Selenium\google authorize.ipynb:

```
var importer = require('../Core');

var approveSelenium = (client) => {
    return client
        .waitForVisible('#scope0 + a', 3000)
        .moveToObject('#scope0')
        .moveToObject('#submit_approve_access')
        .waitForVisible('#submit_approve_access content', 3000)
        .click('#submit_approve_access content')
        .waitForVisible('#code', 3000)
        .getValue('#code');
};

var authorizeSelenium = (authUrl) => {
    var seleniumDocker = null,
        getSeleniumServer = null,
        createWebdriverClient = null,
        client = null, 
        newToken = null;
    console.log('Authorizing: ' + authUrl);
    return importer.interpretAll([
        'selenium docker',
        'set up selenium server',
        'webdriver client node',
        'decrypt password',
        'log in Google'
    ]).then(r => {
        seleniumDocker = eval(r[0].code);
        getSeleniumServer = eval(r[1].code);
        createWebdriverClient = eval(r[2].code);
        eval(r[3].code); // loginGoogle functions use the client.addCommand
        eval(r[4].code);
        return getSeleniumServer('act-selenium')
    })
    .then(() => {
        client = createWebdriverClient('localhost', 4444);
        return client
            .url(authUrl)
            .loginGoogle();
    })
    .then(() => approveSelenium(client))
    .then(code => {
        newToken = code;
        console.log(newToken);
        return newToken;
    });
};
(authorizeSelenium);


```


C:\Users\brian.cullinan\Documents\jupytangular2\Selenium\google timeline.ipynb:

```
var importer = require('../Core');

var client;
var getClient = () => {
    return importer.interpretAll([
        'webdriver client node',
        'decrypt password',
        'log in Google'
    ])
    .then(r => {
        var createWebdriverClient = eval(r[0].code);
        client = createWebdriverClient('localhost', 4444);
        eval(r[1].code);
        eval(r[2].code);
        return client;
    });
};
(getClient);


```


C:\Users\brian.cullinan\Documents\jupytangular2\Selenium\What is Selenium.ipynb:

```
var path = require('path');
var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');
var execCmd;
var getSeleniumServer = (name = 'act-selenium') => {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') { throw err; }
    }
    var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));
    return seleniumDocker(DOCKERFILE)
        .then(() => importer.interpretAll('spawn child process'))
        .then(r => {
        console.log(r);
            execCmd = eval("'use strict';" + r[0].code);
            return execCmd('docker ps -a');
        }).then(r => {
            var build = 'docker build -t ' + name + ' "' 
                + path.dirname(DOCKERFILE) + '"\n'
                + (r[0].indexOf(name) > -1 
                   ? ('docker stop ' + name + '\n') : '')
                + 'docker run --rm --shm-size=2g -d '
                + '--name ' + name + ' ' 
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                // TODO: add profile dir back in when permissions works on windows
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads ' 
                + name + '\n'
                + 'sleep 5\n';
            return execCmd(build)
        });
};
(getSeleniumServer);
```


C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\npm.ipynb:

```
var execSync = require('child_process').execSync;
try {
    require.resolve('npm');
} catch (e) {
    execSync('npm install npm');
}

var npm = (project, args, conf) => {
    var previous = process.cwd();
    process.chdir(project);
    return importer
        .interpretAll('memory-fs rewire')
        .then(r => {
        // set up project path
        return new Promise((resolve, reject) => {
            //if(typeof mockTypescriptFs == 'undefined') {
            //    mockTypescriptFs = eval("'use strict';" + r[0].code);
                // overlay out temp filesystem on top of current filesystem
            //    mockTypescriptFs(project, data);
            //}
            
            var cli = require('npm');
            cli.load(conf, (err) => {
                // handle errors
                if(err) {
                    return reject(err);
                }

                // install module
                cli.commands[args[0]](args.slice(1), (er, data) => {
                    if(er) {
                        return reject(er);
                    }
                    // log errors or data
                    resolve(data);
                });

                cli.on('log', (message) => {
                    // log installation progress
                    console.log(message);
                });
            });
        });
    })
    .then((r => {
        process.chdir(previous);

        return r;
    }));
};
(npm);

```



Find relative paths and includes?

In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;
var importer = require('../Core');

var gulp = require('gulp');
var tap = require('gulp-tap');

var relatives = [];
var packages = [];
var missing = {};
var missingPackages = [];

gulp.task('get relative links', function() {
    return gulp.src(['src/**/*.ts', '!**/*.spec.ts'], {cwd: project})
        .pipe(tap(function (file) {
            var content = file.contents.toString();
            var re = /from\s+["'](.*?)['"]/ig;
            var m;
            while ((m = re.exec(content))) {
                if(m[1].match(/^\./)) {
                    relatives[relatives.length] = file.path + '/' + m[1];
                    var res = path.resolve(path.join(path.dirname(file.path), m[1]));
                    if(!fs.existsSync(res) && !fs.existsSync(res + '.ts')) {
                        var key = path.basename(m[1]);
                        if(typeof missing[key] != 'undefined') {
                            missing[key][missing[key].length] = file.path + '/' + m[1];
                        } else {
                            missing[key] = [file.path + '/' + m[1]];
                        }
                    }
                } else {
                    packages[packages.length] = file.path + '/' + m[1];
                    try {
                        module.paths.unshift(project + '/node_modules');
                        require.resolve(m[1]);
                    } catch (e) {
                        if(missingPackages.indexOf(m[1]) == -1) {
                            missingPackages[missingPackages.length] = file.path + '/' + m[1];
                        }
                    } finally {
                        module.paths.shift();
                    }
                }
            }
        }));
});

var getRelativeLinks = (p) => {
    relatives = [];
    packages = [];
    missing = {};
    missingPackages = [];
    project = p;
    return importer.interpretAll([
        'gulp task to a Promise'
    ])
        .then(r => r[0].runInNewContext({gulp}))
        .then(r => r(['get relative links']))
        .then(r => ({relatives, packages, missing, missingPackages}));
}
module.exports = getRelativeLinks;
getRelativeLinks;



In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/universal') : project;
var importer = require('../Core');

$$.async();
importer.interpretAll([
    'relative paths and includes'
])
.then(r => r[0].runInNEL())
.then(r => r['getRelativeLinks'](project))
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e));

